In [ ]:
%pip install gymnasium stable-baselines3 torch
%pip install stable-baselines3[extra]
%pip install sb3-contrib
%pip install torch torchvision
%pip install numpy protobuf onnx onnxruntime
%pip install onnx


# 強化学習モデルの学習 (main.py)

このセルでは、DQNアルゴリズムを用いて、`CartPole-v1`環境でモデルを学習させます。

In [ ]:
from pettingzoo.test import api_test
from cat_toy_env import CatToyEnv
env_kwargs=dict(render_mode=None, max_steps=1000)

# 1個だけ環境を作る（並列ではなく）
env = CatToyEnv(**env_kwargs)
api_test(env, num_cycles=1000, verbose_progress=False)

In [ ]:
import gymnasium as gym
import torch
from cat_toy_env import CatToyEnv
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import os

In [ ]:
num_iterations = 10000
num_episodes_per_iteration = 1
num_steps_per_episode = 1000
# num_epoches = 1
# num_replays_per_episode = num_epoches * num_episodes_per_iteration * num_steps_per_episode
update_target_steps = 10
replay_interval = 4

In [ ]:
env_kwargs=dict(render_mode=None, max_steps = num_steps_per_episode)
# 1個だけ環境を作る
env_preview = CatToyEnv(**env_kwargs)

obs = env_preview.reset()

# 観測のshapeを確認
print("観測の形:", obs)
print("観測の中身:", obs)
# 学習用環境
env_learning = CatToyEnv(**env_kwargs)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

# CNNを使ったQネットワーク
class DQN(nn.Module):
    def __init__(self, input_shape, output_dim):
        super(DQN, self).__init__()
        c, h, w = input_shape  # (1, H, W)

        self.conv = nn.Sequential(
            nn.Conv2d(c, 2, kernel_size=3, stride=1, padding=1),  # (8, H, W)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                   # (8, H/2, W/2)

            nn.Conv2d(2, 4, kernel_size=3, stride=1, padding=1), # (16, H/2, W/2)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                    # (16, H/4, W/4),                                   # (8, H/2, W/2)

            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                                  
        )

        # 畳み込み後の特徴量サイズを計算
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)
            conv_out = self.conv(dummy_input)
            conv_out_size = conv_out.view(1, -1).size(1)
            print("Conv出力の形:", conv_out.shape)

        # 全結合層
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x)

class DQNAgent:
    def __init__(self, agent_name, env, learning_rate=1e-4, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.agent_name = agent_name  # エージェント名（'cat' または 'toy'）
        self.action_space = env.action_spaces[self.agent_name]  # 各エージェントに対応するアクション空間
        self.state_shape = env.observation_spaces[self.agent_name].shape  # 例: (1, H, W)
        self.model = DQN(self.state_shape, self.action_space.n)
        self.target_model = DQN(self.state_shape, self.action_space.n)

        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.loss_fn = nn.MSELoss()

        self.memory = deque(maxlen=10000)
        self.batch_size = 64
        self.update_target_model()

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def store_experience(self, state, action, reward, next_state, done):
        self.memory.append((
            np.array(state, dtype=np.float32),
            action,
            reward,
            np.array(next_state, dtype=np.float32),
            done
        ))

    def act(self, state):
        if random.random() <= self.epsilon:
            return self.action_space.sample()  # ランダム行動
        state = torch.FloatTensor(state).unsqueeze(0)  # バッチ次元を追加
        q_values = self.model(state)
        return torch.argmax(q_values).item()  # 最大Q値に基づいて行動を選択

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(np.array(states))
        next_states = torch.FloatTensor(np.array(next_states))
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones)

        # Q値の計算
        current_q_values = self.model(states).gather(1, actions.unsqueeze(1))
        next_q_values = self.target_model(next_states).max(1)[0].detach()
        target_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

        # 損失計算とバックプロパゲーション
        loss = self.loss_fn(current_q_values.squeeze(1), target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # εを減少させる
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self, filepath):
        torch.save(self.model.state_dict(), filepath)

    def load_model(self, filepath):
        self.model.load_state_dict(torch.load(filepath))
        self.target_model.load_state_dict(self.model.state_dict())


In [ ]:
def train_dqn(agent_dict, env, num_iterations, num_episodes_per_iteration):
    total_rewards = {agent: 0.0 for agent in env.agents}
    rewards = {agent: 0.0 for agent in env.agents}
    steps = 0
    for iteration in range(num_iterations):
        for episode in range(num_episodes_per_iteration):
            obs = env.reset()
            prev_obs = {agent: obs for agent in env.agents}
            prev_action = {agent: None for agent in env.agents}
            is_last_reward = False

            for agent in env.agent_iter():
                obs, total_reward, terminated, truncated, _ = env.last()
                done = terminated or truncated
                
                if not is_last_reward:
                    rewards = env.rewards.copy()  # 各エージェントの報酬を更新

                if prev_action[agent] is not None:
                    # 前回行動の結果が今回のループで得られたので、ここで保存できる
                    agent_dict[agent].store_experience(
                        prev_obs[agent],         # s
                        prev_action[agent],      # a
                        rewards[agent],      # r (現在のループで得られた報酬)
                        obs,                     # s' (次状態)
                        float(done)              # done
                    )
                    # ここでreplayを行う
                    if env.step_count % replay_interval == 0:
                        for replay_agent in agent_dict.keys():
                            agent_dict[replay_agent].replay()

                if done:
                    action = None  # No action needed if agent is done
                    total_rewards[agent] += total_reward
                    steps += env.step_count
                    print(f"Episode {episode} finished for agent {agent} with reward {total_reward}, {rewards[agent]}, steps {env.step_count}")
                    is_last_reward = True
                else:
                    action = agent_dict[agent].act(obs)

                env.step(action)

                prev_obs[agent] = obs  # 次の状態を更新
                prev_action[agent] = action  # 次の行動を更新

        # ログ出力
        if iteration % update_target_steps == 0:
            print(f"Iteration {iteration}: " + ", ".join([f"{a}: {r / update_target_steps:.2f}" for a, r in total_rewards.items()]), steps / update_target_steps)
            total_rewards = {agent: 0.0 for agent in total_rewards.keys()}
            steps = 0

        # ターゲットネットワーク更新
        if iteration % update_target_steps == 0:
            for agent in agent_dict.values():
                agent.update_target_model()

def evaluate_model(agent_dict, eval_env, n_eval_episodes=10):
    reward_sums = {agent_name: [] for agent_name in agent_dict.keys()}

    for _ in range(n_eval_episodes):
        env = eval_env  # 環境がreset可能で、内部状態が共有でないと仮定
        env.reset()
        episode_rewards = {agent_name: 0.0 for agent_name in agent_dict.keys()}

        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()
            done = termination or truncation

            if done:
                action = None  # 終了したら行動不要
            else:
                action = agent_dict[agent].act(obs)  # 各エージェントに行動させる

            env.step(action)
            episode_rewards[agent] += reward  # 各agentごとに報酬を記録

        for agent_name in reward_sums:
            reward_sums[agent_name].append(episode_rewards[agent_name])

    # 統計量（平均・標準偏差）を返す
    mean_std_rewards = {
        agent: (np.mean(rewards), np.std(rewards))
        for agent, rewards in reward_sums.items()
    }

    return mean_std_rewards

def save_dqn(agent_dict, base_path = "models"):
    os.makedirs(base_path, exist_ok=True)
    for agent_name, agent in agent_dict.items():
        filepath = os.path.join(base_path, f"{agent_name}_model.pth")
        agent.save_model(filepath)

def load_dqn(env, agents = ["cat", "toy"] , base_path = "models"):
    agent_dict = {}
    for agent_name in agents:
        filepath = os.path.join(base_path, f"{agent_name}_model.pth")
        agent = DQNAgent(agent_name, env)
        agent.load_model(filepath)
        agent_dict[agent_name] = agent
    return agent_dict

In [ ]:
# エージェントの作成
agent_dict = {
    agent_name: DQNAgent(agent_name, env_learning)
    for agent_name in env_learning.agents
}


In [ ]:
# 学習
train_dqn(agent_dict, env_learning, num_iterations, num_episodes_per_iteration)


In [ ]:
# 評価用環境
env_kwargs=dict(render_mode="human", max_steps=1000)
env_eval = CatToyEnv(**env_kwargs)

# モデル評価
mean_std_rewards = evaluate_model(agent_dict, env_eval, n_eval_episodes=1)
print(f"mean_reward: {mean_std_rewards['cat']} +/- {mean_std_rewards['toy']}")

In [ ]:
# モデルの保存
save_dqn(agent_dict, "models")

In [ ]:
# 評価用環境
env_kwargs=dict(render_mode="human", max_steps=1000)
env_eval = CatToyEnv(**env_kwargs)

# モデルのロード
loaded_model = load_dqn(env_eval, ["cat", "toy"], "models")

# ロードしたモデルの評価
mean_std_rewards = evaluate_model(loaded_model, env_eval, n_eval_episodes=10)
print(f"mean_reward: {mean_std_rewards['cat']} +/- {mean_std_rewards['toy']}")

In [ ]:
# 入力の2つのTensorを結合
toy = torch.randn(1, 2)
cat = torch.randn(1, 2)
concat_input = torch.cat([toy, cat], dim=1)  # shape: (1, 4)

# エクスポート対象モデル（例: policyネットワーク）
# dummyの環境
env_kwargs=dict(render_mode="human", max_steps=1000)
env_dummy = CatToyEnv(**env_kwargs)

# モデルのロード
loaded_model = load_dqn(env_dummy, ["cat", "toy"], "models")
policy_net = loaded_model["cat"].model  # catエージェントのポリシーネットワークを取得

# ONNXエクスポート
torch.onnx.export(
    policy_net,
    concat_input,  # ← dictではなく単一Tensor
    "cat_dqn_policy.onnx",
    export_params=True,
    opset_version=11,
    input_names=["obs"],
    output_names=["q_values"],
    dynamic_axes={
        "obs": {0: "batch_size"},
        "q_values": {0: "batch_size"}
    }
)


In [ ]:
# 環境のクローズ
env_learning.close()
env_eval.close()

# 学習済みモデルの使用 (play.py)

このセルでは、学習済みのモデルをロードし、`CartPole-v1`環境でエージェントがどのように行動するかを観察します。

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN
import time
from cat_toy_env import CatToyEnv

In [ ]:
env_kwargs=dict(render_mode="", max_steps=1000, cat_speed = 2)

# 環境の作成
env = CatToyEnv(**env_kwargs)

# モデルのロード
model_playing = DQN.load("cat_dqn")

In [ ]:
# エピソードの実行
obs, info = env.reset()
done = False
while not done:
    action, _states = model_playing.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print("観測:", obs)
    done = terminated or truncated
    env.render()  # 環境の描画
    #time.sleep(0.001) # 0.01秒待機

In [ ]:
# 環境のクローズ
env.close()